<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_Generative-Adversarial-Network/blob/main/TF2_0_SGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist

from tensorflow.keras.layers import (Activation, BatchNormalization, Concatenate, Dense, Dropout, Flatten, Input, Lambda, Reshape)
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
#MNIST image
img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)
z_dim = 100

num_classes = 10

In [ ]:
#Data set
class Datasets : 
  def __init__(self, num_labeled) : 
    self. num_labeled = num_labeled

    (self.x_train, self.y_train), (self.x_test, self.y_test) = mnist.load_data()

    def preprocess_imgs(x) : #Norm
      x = (x.astype(np.float32) - 127.5) / 127.5
      x = np.expand_dims(x, axis = 3) # Width X Height X Channel로 확장
      return x

    def preprocess_labels(y) : 
      return y.reshape(-1,1)

    self.x_train = preprocess_imgs(self.x_train)
    self.y_train = preprocess_labels(self.y_train)

    self.x_test = preprocess_imgs(self.x_test)
    self.y_test = preprocess_labels(self.y_test)

  def batch_labeled(self, batch_size ) :
    idx = np.random.randint(0, self.num_labeled, batch_size)
    imgs = self.x_train[idx]
    labels = self.y_train[idx]
    return imgs, labels

  def batch_unlabeled(self, batch_size) : 
    idx = np.random.randint(self.num_labeled, self.x_train.shape[0], batch_size) 
    imgs = self.x_train[idx]
    return imgs

  def training_set(self) : 
    x_train = self.x_train[range(self.num_labeled)]
    y_train = self.y_train[range(self.num_labeled)]
    return x_train, y_train

  def test_set(self) : 
    return self.x_test, self.y_test



In [ ]:
num_labeled = 100
dataset = Datasets(num_labeled)

In [ ]:
#Designing Generator

def build_generator(z_dim) : 
  model = Sequential()
  model.add(Dense(256 * 7 * 7, input_dim = z_dim))
  model.add(Reshape((7,7,256)))

  model.add(Conv2DTranspose(128, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.01))

  model.add(Conv2DTranspose(64, kernel_size = 3, strides = 1, padding = 'same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.01))

  model.add(Conv2DTranspose(1, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(Activation('tanh'))

  return model

In [ ]:
#Designing Discriminator

def build_discriminator_net(img_shape) : 
  model = Sequential()
  model.add(Conv2D(32, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(LeakyReLU(alpha = 0.01))

  model.add(Conv2D(64, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(LeakyReLU(alpha = 0.01))

  model.add(Conv2D(128, kernel_size = 3, strides = 2, padding = 'same'))
  model.add(LeakyReLU(alpha = 0.01))

  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(num_classes))
  return model

In [ ]:
#Discriminator of supervised learning

def build_discriminator_supervised(discriminator_net) : 
  model = Sequential()
  model.add(discriminator_net)
  model.add(Activation('softmax'))

  return model

In [ ]:
#Discriminator of semi-supervised learning
#input으로 들어온 이미지가 진짜일지 가짜일지를 predict(x) 함수를 통해 구함
#이미지가 가짜일 경우 x값이 전체적으로 작음 -> prediction값이 1에 가깝게 나옴, 반대로 이미지가 진짜일 경우 x값이 전체적으로 크고 prediction값이 0에 가깝게 나옴
#즉 prediction은 가짜일 확률로 보면됨

def build_discriminator_unsupervised(discriminator_net) : 
  model = Sequential()
  model.add(discriminator_net)

  def predict(x) : 
    prediction = 1.0 - (1.0 / (K.sum(K.exp(x), axis = -1, keepdims = True) + 1.0))
    return prediction

  model.add(Lambda(predict))
  return model

In [ ]:
#build GAN
# [1]Supervised-learning기법으로 이미 라벨링된 이미지를 통해 학습시킴
# [2]Unsupervised-learning기법으로 [1]로 학습시킨 Discriminator가 진짜이미지와 가짜이미지를 구분하도록 학습
# [3]Generator만 학습하기 위해 Discriminator 학습을 끔
# [4]z_dim기반으로 Generator가 만든 이미지를 [2]까지 학습시킨 Discriminator를 통해, 참-거짓을 구분하고 Generator 학습 
# 단점은 Generator가 학습할때 Reference로 삼는 Disctrimiator의 예측값이 반드시 참이라는 보장이 없음 → Discriminator 오차가 누적된 Geneator 오차 발생 가능

def build_gan(generator, discriminator) : 
  model = Sequential()
  model.add(generator)
  model.add(discriminator)

  return model


In [ ]:
discriminator_net = build_discriminator_net(img_shape) 
discriminator_supervised = build_discriminator_supervised(discriminator_net) #[1]
discriminator_supervised.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = Adam(learning_rate = 0.0003))

discriminator_unsupervised = build_discriminator_unsupervised(discriminator_net) #[2]
discriminator_unsupervised.compile(loss = 'binary_crossentropy', optimizer=Adam())

generator = build_generator(z_dim)
discriminator_unsupervised.trainable = False #[3]

gan = build_gan(generator, discriminator_unsupervised) #[4]
gan.compile(loss = 'binary_crossentropy', optimizer = Adam())

In [ ]:
#Pipeline
supervised_losses = []
iteration_checkpoints = []


def train(iterations, batch_size, sample_interval):

    # 진짜 이미지의 레이블: 모두 1
    real = np.ones((batch_size, 1))

    # 가짜 이미지의 레이블: 모두 0
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):

        # -------------------------
        #  판별자 훈련
        # -------------------------

        # 레이블된 샘플을 가져옵니다.
        imgs, labels = dataset.batch_labeled(batch_size)

        # 레이블을 원-핫 인코딩합니다.
        labels = to_categorical(labels, num_classes=num_classes)

        # 레이블이 없는 샘플을 가져옵니다.
        imgs_unlabeled = dataset.batch_unlabeled(batch_size)

        # 가짜 이미지의 배치를 생성합니다.
        z = np.random.normal(0, 1, (batch_size, z_dim))
        gen_imgs = generator.predict(z)

        # 레이블된 진짜 샘플에서 훈련합니다.
        d_loss_supervised, accuracy = discriminator_supervised.train_on_batch(imgs, labels)

        # 레이블이 없는 진짜 샘플에서 훈련합니다.
        d_loss_real = discriminator_unsupervised.train_on_batch(
            imgs_unlabeled, real)

        # 가짜 샘플에서 훈련합니다.
        d_loss_fake = discriminator_unsupervised.train_on_batch(gen_imgs, fake)

        d_loss_unsupervised = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  생성자 훈련
        # ---------------------

        # 가짜 이미지의 배치를 생성합니다.
        z = np.random.normal(0, 1, (batch_size, z_dim))
        gen_imgs = generator.predict(z)

        # 생성자를 훈련합니다.
        g_loss = gan.train_on_batch(z, np.ones((batch_size, 1)))

        if (iteration + 1) % sample_interval == 0:

            # 훈련이 끝난 후 그래프를 그리기 위해 판별자의 지도 학습 분류 손실을 기록합니다.
            supervised_losses.append(d_loss_supervised)
            iteration_checkpoints.append(iteration + 1)

            # 훈련 과정을 출력합니다.
            print(
                "%d [D loss supervised: %.4f, acc.: %.2f%%] [D loss unsupervised: %.4f] [G loss: %f]"
                % (iteration + 1, d_loss_supervised, 100 * accuracy,
                   d_loss_unsupervised, g_loss))

In [ ]:
#Running
iterations = 8000
batch_size = 32
sample_interval = 100

train(iterations, batch_size, sample_interval)

In [ ]:
losses = np.array(supervised_losses)

# 판별자의 지도 학습 손실을 그립니다.
plt.figure(figsize=(15, 5))
plt.plot(iteration_checkpoints, losses, label="Discriminator loss")

plt.xticks(iteration_checkpoints, rotation=90)

plt.title("Discriminator – Supervised Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
x, y = dataset.training_set()
y = to_categorical(y, num_classes=num_classes)

# 훈련 세트에서 분류 정확도 계산
_, accuracy = discriminator_supervised.evaluate(x, y)
print("Training Accuracy: %.2f%%" % (100 * accuracy))

In [ ]:
x, y = dataset.test_set()
y = to_categorical(y, num_classes=num_classes)

# 테스트 세트에서 분류 정확도 계산
_, accuracy = discriminator_supervised.evaluate(x, y)
print("Test Accuracy: %.2f%%" % (100 * accuracy))

In [ ]:
# SGAN 판별자와 같은 네트워크 구조를 가진 지도 학습 분류기
mnist_classifier = build_discriminator_supervised(build_discriminator_net(img_shape))
mnist_classifier.compile(loss='categorical_crossentropy',
                         metrics=['accuracy'],
                         optimizer=Adam())

In [ ]:
imgs, labels = dataset.training_set()

# 레이블을 원-핫 인코딩합니다.
labels = to_categorical(labels, num_classes=num_classes)

# 분류기를 훈련합니다.
training = mnist_classifier.fit(x=imgs,
                                y=labels,
                                batch_size=32,
                                epochs=30,
                                verbose=1)
losses = training.history['loss']
accuracies = training.history['accuracy']

In [ ]:
# 분류 손실을 그립니다
plt.figure(figsize=(10, 5))
plt.plot(np.array(losses), label="Loss")
plt.title("Classification Loss")
plt.legend()
plt.show()

In [ ]:
# 분류 정확도를 그립니다.
plt.figure(figsize=(10, 5))
plt.plot(np.array(accuracies), label="Accuracy")
plt.title("Classification Accuracy")
plt.legend()
plt.show()

In [ ]:
x, y = dataset.training_set()
y = to_categorical(y, num_classes=num_classes)

# 훈련 세트에 대한 분류 정확도를 계산합니다.
_, accuracy = mnist_classifier.evaluate(x, y)
print("Training Accuracy: %.2f%%" % (100 * accuracy))

In [ ]:
x, y = dataset.test_set()
y = to_categorical(y, num_classes=num_classes)

# 테스트 세트에 대한 분류 정확도를 계산합니다.
_, accuracy = mnist_classifier.evaluate(x, y)
print("Test Accuracy: %.2f%%" % (100 * accuracy))